A\*算法在十五数码游戏中的应用：有15个编号为1-15的棋子放在4\*4的方格棋盘上，棋盘上有一格是空的，空格周围的棋子可以走进空格，初始棋局与目标棋局分别如下所示，请问如何走步可以把初始棋局变换为目标棋局？
start=[[11,9,4,5],[1,3, None,12],[7,25,8,6],[13,2,10,14]]
end=[[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,None]]
要求如下
1. 使用A*搜索算法，其中估价函数为：
f（n）=g（n）+h（n），其中g（n）为结点n的深度，h（n）为错位牌数。
2. 十五数码问题中每个状态的属性至少应包括：
数码的排列；
当前状态的深度；
当前状态到目标状态的最小估计；
当前状态的父节点状态。
3. 算符共4个：空格上方、下方、左侧、右侧的数码移入空格。
4. open表中为新扩展的状态，cosed表中为已扩展过的状态，open表中的状态应按估价函数的大小排序。
输出结果为从初始状态到目标状态的所有步骤。

In [ ]:
class State:
    def __init__(self, map, depth, parent=None):
        self.map = map                  # 当前棋盘状态
        self.depth = depth               # 当前状态的深度
        self.parent = parent             # 父节点
        self.empty_loc = self.find_empty()  # 空格的位置
        self.predict = self.depth + self.misplaced_tiles(end)  # 估价函数f(n) = g(n) + h(n)
        
    def find_empty(self):
        # 找到空格的位置
        for i in range(len(self.map)):
            for j in range(len(self.map[0])):
                if self.map[i][j] is None:
                    return [i, j]

    def misplaced_tiles(self, end):
        # 计算错位牌数 h(n)
        mis = 0
        for i in range(len(self.map)):
            for j in range(len(self.map[0])):
                if self.map[i][j] != end[i][j] and self.map[i][j] is not None:
                    mis += 1
        return mis

    def __eq__(self, other):
        return self.map == other.map
    
    def __lt__(self, other):
   
        return self.predict < other.predict
    
    def print_map(self) :
        return self.map
    
    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.map))
        


In [13]:

# 移动函数
def up_move(current):
    loc = current.empty_loc
    if loc[0] > 0:
        # 生成新状态
        new_map = [row[:] for row in current.map]
        new_map[loc[0]][loc[1]], new_map[loc[0] - 1][loc[1]] = new_map[loc[0] - 1][loc[1]], new_map[loc[0]][loc[1]]
        return State(new_map, current.depth + 1, current)
    return None

def down_move(current):
    loc = current.empty_loc
    if loc[0] < 3:
        new_map = [row[:] for row in current.map]
        new_map[loc[0]][loc[1]], new_map[loc[0] + 1][loc[1]] = new_map[loc[0] + 1][loc[1]], new_map[loc[0]][loc[1]]
        return State(new_map, current.depth + 1, current)
    return None

def left_move(current):
    loc = current.empty_loc
    if loc[1] > 0:
        new_map = [row[:] for row in current.map]
        new_map[loc[0]][loc[1]], new_map[loc[0]][loc[1] - 1] = new_map[loc[0]][loc[1] - 1], new_map[loc[0]][loc[1]]
        return State(new_map, current.depth + 1, current)
    return None

def right_move(current):
    loc = current.empty_loc
    if loc[1] < 3:
        new_map = [row[:] for row in current.map]
        new_map[loc[0]][loc[1]], new_map[loc[0]][loc[1] + 1] = new_map[loc[0]][loc[1] + 1], new_map[loc[0]][loc[1]]
        return State(new_map, current.depth + 1, current)
    return None




In [14]:
# A* 搜索主函数
def search(start, end):
    start_state = State(start, 0)
    print(start_state.print_map())
    end_state = end
    open_list = [start_state]
    closed_list = []

    while open_list:
        # 从 open_list 中找到 f(n) 最小的节点
        open_list.sort(key=lambda x: x.predict)
        current_state = open_list.pop(0)
        
        

        # 检查是否到达目标状态
        if current_state.map == end_state:
            path = []
            while current_state:
                path.append(current_state.map)
                current_state = current_state.parent
            return path[::-1]

        # 将当前状态加入 closed_list
        closed_list.append(current_state.map)
 

        # 扩展新状态
        for move in [up_move, down_move, left_move, right_move]:
            new_state = move(current_state)
            if new_state and new_state.map not in closed_list:
                open_list.append(new_state)

    return None  # 如果没有找到解决方案则返回 None

# 初始化 start 和 end 状态
start = [[5, 1, 2, 4], [9, 6, 3,8], [13, 15, 10, 11], [None, 14, 7, 12]]
end = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, None]]

# 执行搜索
path = search(start, end)
if path:
    for step, state in enumerate(path):
        print(f"Step {step}:")
        for row in state:
            print(row)
else:
    print("No solution found.")

[[5, 1, 2, 4], [9, 6, 3, 8], [13, 15, 10, 11], [None, 14, 7, 12]]
Step 0:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 15, 10, 11]
[None, 14, 7, 12]
Step 1:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 15, 10, 11]
[14, None, 7, 12]
Step 2:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, None, 10, 11]
[14, 15, 7, 12]
Step 3:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 10, None, 11]
[14, 15, 7, 12]
Step 4:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 10, 7, 11]
[14, 15, None, 12]
Step 5:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 10, 7, 11]
[14, None, 15, 12]
Step 6:
[5, 1, 2, 4]
[9, 6, 3, 8]
[13, 10, 7, 11]
[None, 14, 15, 12]
Step 7:
[5, 1, 2, 4]
[9, 6, 3, 8]
[None, 10, 7, 11]
[13, 14, 15, 12]
Step 8:
[5, 1, 2, 4]
[None, 6, 3, 8]
[9, 10, 7, 11]
[13, 14, 15, 12]
Step 9:
[None, 1, 2, 4]
[5, 6, 3, 8]
[9, 10, 7, 11]
[13, 14, 15, 12]
Step 10:
[1, None, 2, 4]
[5, 6, 3, 8]
[9, 10, 7, 11]
[13, 14, 15, 12]
Step 11:
[1, 2, None, 4]
[5, 6, 3, 8]
[9, 10, 7, 11]
[13, 14, 15, 12]
Step 12:
[1, 2, 3, 4]
[5, 6, None, 8]
[9, 10, 7, 11]
[13, 14, 15, 12]
Step 13:
[1, 2, 3, 4]
[5, 6, 7, 8]